<h1><b>Crawling & Scrawling Data</h1></b> 

In [1]:
import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import numpy as np
import requests 
import math
import re
import time

In [2]:
def generate_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")    
    #Crawl-delay: 3
    time.sleep(3)
    links=soup.find_all("a")
    list_of_links=[]
    for i,a in enumerate(links):
        string=a.get('href')
        time.sleep(3)
        if ("/draft/NBA_" in string)  :
            suffix=string.replace("/draft","")
            list_of_links.append(url+suffix)  
            if "1970" in string:
                break                  
    return list_of_links        

In [3]:
def get_properties(url):
    pk=[]
    team=[]
    names=[]
    college=[]
    seasons=[]
    games=[]
    minutes=[]
    points=[]
    ttlribounds=[]
    asists=[]
    FGper=[]
    threePper=[]
    FTper=[]
    MP=[]
    PntperGame=[]
    ttlriboundsperGame=[]
    asistsperGame=[]
    WS=[]
    WSminutes=[]
    BPM=[]
    vorp=[]
    player_links=[]
    years=[]
    response = requests.get(url)
    time.sleep(3)
    soup = BeautifulSoup(response.content, "html.parser")
    #the number of <tr> within <table> 
    rows=soup.table.tbody.find_all("tr")
    #it takes 2 rounds inorder to compleate the draft 
    #each year has diffarent number of players to pick from --> difarent number of <tr> each <table>
    rounds=math.floor((len(rows)/2)-1)     
    for i,r in enumerate(rows):
                years.append(int(url[47:51]))
                #at some years like 2001 there are missing rows, "Minnesota forfeited the 29th overall pick."
                if r.find("td",attrs={"class":"center"}):
                    continue
                if (i < rounds or i>rounds+1) and (r.find_all("a",attrs={"href":re.compile("player")})) :
                    player_links.append(url[:36]+r.find_all("a",attrs={"href":re.compile("player")})[0].get("href")) #players_links 
                    l = r.find_all("td")
                    pk.append(l[0].string)
                    team.append(l[1].string)
                    names.append(l[2].string)
                    college.append(l[3].string)
                    seasons.append(l[4].string)
                    games.append(l[5].string)
                    minutes.append(l[6].string)
                    points.append(l[7].string)
                    ttlribounds.append(l[8].string)
                    asists.append(l[9].string)
                    FGper.append(l[10].string)
                    threePper.append(l[11].string)
                    FTper.append(l[12].string)
                    MP.append(l[13].string)
                    PntperGame.append(l[14].string)
                    ttlriboundsperGame.append(l[15].string)
                    asistsperGame.append(l[16].string)
                    WS.append(l[17].string)
                    WSminutes.append(l[18].string)
                    BPM.append(l[19].string)
                    vorp.append(l[20].string)
                        
                                                 
    return  pk,team, names, college ,seasons ,games ,minutes ,points, ttlribounds, asists, FGper, threePper, FTper, MP, PntperGame, ttlriboundsperGame, asistsperGame, WS, WSminutes, BPM ,vorp,player_links,years

In [4]:
def generate_player_data(df):
    
    players_links=[]
    players_names=[]
    players_height=[]
    players_weight=[]
    players_position=[]
    
    
    players_names.extend(df['Names'].tolist())
    players_links.extend(df['player page'].tolist())
    
        
    #all players positions
    for link in players_links:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, "html.parser")
        try1 = soup.find("strong",string=re.compile("Position:")).next.next.split("  ")[1].rstrip()
        players_position.append(try1)
        
        #Gets Height
        res1=soup.body.find_all("span")
        
        
        if ')' in res1[9].text:  #Some players don't have full data
            players_height.append("0")
            players_weight.append("0")
            continue
        
        players_height.append(res1[9].text)
        
        #Gets Weight
        res2=soup.body.find_all("span")
        players_weight.append(res2[10].text)
        
        
    #turns Height 6-5 to 6.5
    for i in range(len(players_height)):
        players_height[i] = players_height[i].replace("-", ".")

    #Creat new dataframe with data
    data=pd.DataFrame(list(zip(players_names, players_height, players_weight,players_position)), columns=["Names", "Height", "Weight","Position"])
    
    #clean Height & Weight
    data['Height'] = data['Height'].astype(float)
    data['Height'] = 30.01818181818182 * data['Height']
    data['Weight'] = data['Weight'].str.replace('lb', '')
    data['Weight'] = data['Weight'].astype(float)
    data['Weight'] = 0.453592 * data['Weight']
    
    
    #split positions
    new = data["Position"].str.split("and", n = 1, expand = True)
    data["First Position"]= new[0]
    


    #Cleans first position from ',' or 'and'
    new2 = data["First Position"].str.split(",", n = 1, expand = True)
    data["First Position"]= new2[0]
    
    #Cleans first position from ' '
    data["First Position"] = data["First Position"].str.strip()
    
    #Merge with original data
    df_final = pd.merge(df, data, how='inner', on = 'Names')
    
    
    return df_final


In [5]:
def clean_zero_pages_names(df):
    df = df[df['Height'] != 0]
    df = df.drop(['player page','Names','Position'], axis=1)
    
    
    return df

<h2><b> crawling links </b></h2>

Robots.txt request a 3 sec crawl-delay and the mount of Data thet we needed for our project therefore the Data collection process can take nearly an hour!

In [6]:
links=[]
links.extend(generate_links("https://www.basketball-reference.com/draft"))

In [10]:
links

['https://www.basketball-reference.com/draft/NBA_2021.html',
 'https://www.basketball-reference.com/draft/NBA_2020.html',
 'https://www.basketball-reference.com/draft/NBA_2019.html',
 'https://www.basketball-reference.com/draft/NBA_2018.html',
 'https://www.basketball-reference.com/draft/NBA_2017.html',
 'https://www.basketball-reference.com/draft/NBA_2016.html',
 'https://www.basketball-reference.com/draft/NBA_2015.html',
 'https://www.basketball-reference.com/draft/NBA_2014.html',
 'https://www.basketball-reference.com/draft/NBA_2013.html',
 'https://www.basketball-reference.com/draft/NBA_2012.html',
 'https://www.basketball-reference.com/draft/NBA_2011.html',
 'https://www.basketball-reference.com/draft/NBA_2010.html',
 'https://www.basketball-reference.com/draft/NBA_2009.html',
 'https://www.basketball-reference.com/draft/NBA_2008.html',
 'https://www.basketball-reference.com/draft/NBA_2007.html',
 'https://www.basketball-reference.com/draft/NBA_2006.html',
 'https://www.basketball

<b> collecting properties and arrange in DataFrame </b>

In [8]:
df = pd.DataFrame()
df2 = pd.DataFrame()
for i,link in enumerate(links) :
    pk,team, names, college ,seasons ,games ,minutes ,points, ttlribounds, asists, FGper, threePper, FTper, MP, PntperGame, ttlriboundsperGame, asistsperGame, WS, WSminutes, BPM ,vorp,player_links,years = get_properties(link)
    df=pd.DataFrame(list(zip(pk,team,player_links,years,names, college,seasons ,games ,minutes ,points, ttlribounds, asists, FGper, threePper, FTper, MP, PntperGame, ttlriboundsperGame, asistsperGame, WS, WSminutes, BPM ,vorp)), columns=['Overall Pick','Teams','player page','year', 'Names','College','Seasons in NBA', '# Games','Minutes Played','Points','Total Rebounds','Assists','Field Goal %','3 Point Field Goal %', 'Free Throw %', 'Minutes Played per Game','Points per Game', 'Total Rebounds per Game', 'Assists per Game','Win Shares','Win Shares per 48 Minutes','Box +-','VORP'])
    df2=pd.concat([df2,df],axis=0)

In [11]:
df2

,Overall Pick,Teams,player page,year,Names,College,Seasons in NBA,# Games,Minutes Played,Points,...,3 Point Field Goal %,Free Throw %,Minutes Played per Game,Points per Game,Total Rebounds per Game,Assists per Game,Win Shares,Win Shares per 48 Minutes,Box +-,VORP
0,1,DET,https://www.basketball-reference.com/players/c...,2021,Cade Cunningham,Oklahoma State,1,40,1293,642,...,.331,.860,32.3,16.1,5.5,5.4,-0.3,-.013,-1.7,0.1
1,2,HOU,https://www.basketball-reference.com/players/g...,2021,Jalen Green,None,1,34,1021,485,...,.290,.825,30.0,14.3,3.3,2.2,-1.0,-.049,-5.7,-1.0
2,3,CLE,https://www.basketball-reference.com/players/m...,2021,Evan Mobley,USC,1,42,1447,634,...,.300,.662,34.5,15.1,8.2,2.6,3.7,.121,1.1,1.1
3,4,TOR,https://www.basketball-reference.com/players/b...,2021,Scottie Barnes,Florida State,1,39,1405,579,...,.308,.723,36.0,14.8,7.8,3.5,3.1,.106,0.1,0.7
4,5,ORL,https://www.basketball-reference.com/players/s...,2021,Jalen Suggs,Gonzaga,1,30,846,372,...,.252,.762,28.2,12.4,3.7,4.0,-1.1,-.065,-5.4,-0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,235,POR,https://www.basketball-reference.com/players/b...,1970,Bruce Butchko,Southern Illinois,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
233,236,CLE,https://www.basketball-reference.com/players/c...,1970,Emmanuel Cannon,Grambling State University,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
234,237,SDR,https://www.basketball-reference.com/players/e...,1970,Rick Erickson,Washington State,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
235,238,CLE,https://www.basketball-reference.com/players/w...,1970,Allen Waller,Saint Mary of the Plains College,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


<b>the pre-processed DataFrame</b>

<b>cleanning and processing Data</b>

In [12]:
df2.dropna(axis=0,inplace=True)
df3=generate_player_data(df2.copy())
df4 = clean_zero_pages_names(df3.copy())

In [13]:
df3

,Overall Pick,Teams,player page,year,Names,College,Seasons in NBA,# Games,Minutes Played,Points,...,Total Rebounds per Game,Assists per Game,Win Shares,Win Shares per 48 Minutes,Box +-,VORP,Height,Weight,Position,First Position
0,1,DET,https://www.basketball-reference.com/players/c...,2021,Cade Cunningham,Oklahoma State,1,40,1293,642,...,5.5,5.4,-0.3,-.013,-1.7,0.1,198.120000,99.790240,Point Guard,Point Guard
1,3,CLE,https://www.basketball-reference.com/players/m...,2021,Evan Mobley,USC,1,42,1447,634,...,8.2,2.6,3.7,.121,1.1,1.1,210.127273,97.522280,Power Forward,Power Forward
2,4,TOR,https://www.basketball-reference.com/players/b...,2021,Scottie Barnes,Florida State,1,39,1405,579,...,7.8,3.5,3.1,.106,0.1,0.7,207.125455,102.965384,Power Forward,Power Forward
3,5,ORL,https://www.basketball-reference.com/players/s...,2021,Jalen Suggs,Gonzaga,1,30,846,372,...,3.7,4.0,-1.1,-.065,-5.4,-0.7,192.116364,92.986360,Point Guard,Point Guard
4,8,ORL,https://www.basketball-reference.com/players/w...,2021,Franz Wagner,Michigan,1,51,1648,801,...,4.6,2.9,2.6,.077,-0.9,0.5,207.125455,102.058200,Small Forward,Small Forward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2060,40,SEA,https://www.basketball-reference.com/players/h...,1970,Gar Heard,Oklahoma,11,787,19571,6828,...,7.5,1.6,27.0,.066,-0.4,6.3,198.120000,99.336648,Power Forward,Power Forward
2061,103,SDR,https://www.basketball-reference.com/players/p...,1970,Billy Paultz,St. John's,9,637,13630,5432,...,5.6,1.6,26.3,.093,-0.2,6.3,183.411091,106.594120,Center and Power Forward,Center
2062,106,BOS,https://www.basketball-reference.com/players/s...,1970,Charlie Scott,UNC,8,560,19278,10037,...,3.6,4.8,26.9,.067,-1.5,2.1,0.000000,0.000000,Shooting Guard and Point Guard,Shooting Guard
2063,122,DET,https://www.basketball-reference.com/players/i...,1970,Dan Issel,Kentucky,9,718,22342,14659,...,7.9,2.5,82.3,.177,2.3,24.3,207.125455,106.594120,Center and Power Forward,Center


In [14]:
df4

,Overall Pick,Teams,year,College,Seasons in NBA,# Games,Minutes Played,Points,Total Rebounds,Assists,...,Points per Game,Total Rebounds per Game,Assists per Game,Win Shares,Win Shares per 48 Minutes,Box +-,VORP,Height,Weight,First Position
0,1,DET,2021,Oklahoma State,1,40,1293,642,219,214,...,16.1,5.5,5.4,-0.3,-.013,-1.7,0.1,198.120000,99.790240,Point Guard
1,3,CLE,2021,USC,1,42,1447,634,343,109,...,15.1,8.2,2.6,3.7,.121,1.1,1.1,210.127273,97.522280,Power Forward
2,4,TOR,2021,Florida State,1,39,1405,579,303,137,...,14.8,7.8,3.5,3.1,.106,0.1,0.7,207.125455,102.965384,Power Forward
3,5,ORL,2021,Gonzaga,1,30,846,372,110,120,...,12.4,3.7,4.0,-1.1,-.065,-5.4,-0.7,192.116364,92.986360,Point Guard
4,8,ORL,2021,Michigan,1,51,1648,801,236,150,...,15.7,4.6,2.9,2.6,.077,-0.9,0.5,207.125455,102.058200,Small Forward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059,19,CIN,1970,Texas-El Paso,13,876,31159,16481,2046,6476,...,18.8,2.3,7.4,83.4,.128,0.4,12.9,183.110909,68.038800,Point Guard
2060,40,SEA,1970,Oklahoma,11,787,19571,6828,5876,1220,...,8.7,7.5,1.6,27.0,.066,-0.4,6.3,198.120000,99.336648,Power Forward
2061,103,SDR,1970,St. John's,9,637,13630,5432,3553,1043,...,8.5,5.6,1.6,26.3,.093,-0.2,6.3,183.411091,106.594120,Center
2063,122,DET,1970,Kentucky,9,718,22342,14659,5707,1804,...,20.4,7.9,2.5,82.3,.177,2.3,24.3,207.125455,106.594120,Center


In [16]:
df4.to_csv("machine_learning_data.csv")

In [15]:
df4.to_csv("EDA_data.csv")

<h3><b>collecting salaries</b></h3>

<p>inorder to collect salaries from the datasource wi nedd to crawl each player_page , searching for the first year at the salaries table in it. we vlidate that the salary is ctually the first one by matching the draft year from dataframe and the salary year from player_page </p>

In [ ]:
def generate_first_salary(df):
    salaries=[]
    for i,row in df.iterrows():
        response = requests.get(row[2])
        time.sleep(3)
        soup = BeautifulSoup(response.content, "html.parser")      
        l=soup.find_all("div")
        time.sleep(3)
        li=str(l[0]).split("<tr >")
        for k in li:
                flag=0
                if (str(row[3]) in k) and ("csk=" in k):
                    for i in k.split("csk=",2)[len(k.split("csk=",2))-1][:-20].split():
                        if i.strip('""').isdigit() :
                            num=int(i.strip('""'))
                            if num>100000:
                                salaries.append(num)
                                flag=1
                            break
                    break
        if flag==0:
            salaries.append(float("NaN"))       
    return salaries

In [ ]:
li=[]
li.extend(generate_first_salary(df3.copy()))

In [ ]:
df5["salary"]=li

In [ ]:
df5

In [ ]:
df5.dropna(axis=0).to_csv("Full_Data_with_salaries.csv")